In [1]:
import logging
import os

# set the directory to the location of the script
try:
    os.chdir("..")
    target_directory = os.getenv(
        "TARGET_DIRECTORY", os.getcwd()
    )  # Use environment variable if available
    if os.path.exists(target_directory):
        os.chdir(target_directory)
        print(f"Changed directory to: {os.getcwd()}")
        logging.info(f"Successfully changed directory to: {os.getcwd()}")
    else:
        logging.error(f"Directory does not exist: {target_directory}")
except Exception as e:
    logging.exception(f"An error occurred while changing directory: {e}")

Changed directory to: c:\Users\pablosal\Desktop\gbb-ai-audio-agent


### Access Operational DB (Post Analyics) -> CosmosDB

In [2]:
from src.cosmosdb.manager import CosmosDBMongoCoreManager

cosmos_manager = CosmosDBMongoCoreManager(
    connection_string=os.getenv("AZURE_COSMOS_CONNECTION_STRING"),
    database_name=os.getenv("AZURE_COSMOS_DB_DATABASE_NAME"),
    collection_name=os.getenv("AZURE_COSMOS_DB_COLLECTION_NAME"),
)

c:\Users\pablosal\Desktop\gbb-ai-audio-agent\src\cosmosdb\manager.py:31: UserWarning: You appear to be connected to a CosmosDB cluster. For more information regarding feature compatibility and support please visit https://www.mongodb.com/supportability/cosmosdb
  self.client = pymongo.MongoClient(connection_string)


In [8]:
query = {"session_id": "a23455d6"}
document = cosmos_manager.read_document(query)

if document:
    print("Found session:", document)
else:
    print("No session found.")

ERROR:src.cosmosdb.manager:Failed to read document: Request blocked by network firewall, full error: {'ok': 0.0, 'errmsg': 'Request blocked by network firewall', 'code': 18, 'codeName': 'AuthenticationFailed'}, Timeout: 30s, Topology Description: <TopologyDescription id: 68361aaeb71b114712449cdb, topology_type: ReplicaSetNoPrimary, servers: [<ServerDescription ('cosmosdb-ai-factory-westus2.mongo.cosmos.azure.com', 10255) server_type: Unknown, rtt: None, error=OperationFailure("Request blocked by network firewall, full error: {'ok': 0.0, 'errmsg': 'Request blocked by network firewall', 'code': 18, 'codeName': 'AuthenticationFailed'}")>]>


No session found.


## Access Cache (redis) 

In [ ]:
from rtagents.RTMedAgent.backend.orchestration.conversation_state import (
    ConversationManager,
)
from src.redis.manager import AzureRedisManager

redis_manager = AzureRedisManager()

2025-05-27 15:05:31,931 - micro - MainProcess - INFO     Azure Redis connection initialized. (manager.py:__init__:41)
INFO:micro:Azure Redis connection initialized.


In [16]:
cm = ConversationManager.from_redis("a23455d6", redis_manager)

2025-05-27 15:06:15,372 - micro - MainProcess - INFO     Restored session a23455d6: 21 msgs, ctx keys=['authenticated', 'latency_roundtrip', 'phone_number', 'patient_dob', 'patient_id', 'patient_name'] (conversation_state.py:from_redis:41)
INFO:micro:Restored session a23455d6: 21 msgs, ctx keys=['authenticated', 'latency_roundtrip', 'phone_number', 'patient_dob', 'patient_id', 'patient_name']


Templates found: ['voice_agent_authentication.jinja', 'voice_agent_system.jinja']


In [17]:
cm.hist[:100]

[{'role': 'system',
  'content': '\n\n"""ROLE & CONTEXT"""\nYou are RTMedAgent, a HIPAA‑compliant, real‑time healthcare voice assistant\nserving patients of XYZ Health Clinic after multi‑factor authentication.\n\n"""CALL METADATA"""\n• patient_id: P54321\n• patient_name: Alice Brown\n• patient_dob: None\n• patient_phone_number: 5552971078\n\n"""DATA VALIDATION"""\nIf any CALL METADATA field above (patient_name, patient_dob, or patient_phone_number) is missing, politely ask the patient to provide or confirm the missing information before proceeding.\n\n"""INTENT ONTOLOGY (MUST CHOOSE ONE)"""\n1. schedule_appointment\n2. change_appointment\n3. cancel_appointment\n4. refill_prescription\n5. fill_new_prescription\n6. get_current_prescriptions\n7. lookup_medication_info\n8. lookup_side_effects\n9. check_drug_interactions\n10. evaluate_prior_authorization\n11. transfer_medication_pharmacy\n12. insurance_billing_question\n13. escalate_emergency\n14. general_health_question\n15. unknown / need

In [18]:
lat = cm.get_context("latency_roundtrip", {})

for stage, entries in lat.items():
    # extract all the durations
    durations = [e["dur"] for e in entries]

    # compute summary stats
    count = len(durations)
    average = sum(durations) / count if count else 0
    minimum = min(durations) if count else 0
    maximum = max(durations) if count else 0

    # pretty-print
    print(
        f"⏱️ Stage “{stage}” — "
        f"calls: {count}, "
        f"avg: {average:.3f}s, "
        f"min: {minimum:.3f}s, "
        f"max: {maximum:.3f}s"
    )

⏱️ Stage “tts” — calls: 13, avg: 0.001s, min: 0.001s, max: 0.004s
⏱️ Stage “processing” — calls: 5, avg: 2.720s, min: 0.679s, max: 5.468s


## Call Agents

In [ ]:
from rtagents.RTMedAgent.backend.agents.base import RTAgent

In [ ]:
agent = RTAgent(
    config_path="rtagents/RTMedAgent/backend/agents/agent_store/auth_agent.yaml",
    session_id="demo123",
)

2025-05-27 15:06:36,590 - rt_agent - MainProcess - INFO     Loaded agent 'AuthAgent' | org='GBB AI' | desc='Handles caller authentication, then hands off to clinical…' | model=gpt-4o-standard | prompt=voice_agent_authentication.jinja | tools=['authenticate_user'] | session=demo123 (base.py:_log_loaded_summary:143)
INFO:rt_agent:Loaded agent 'AuthAgent' | org='GBB AI' | desc='Handles caller authentication, then hands off to clinical…' | model=gpt-4o-standard | prompt=voice_agent_authentication.jinja | tools=['authenticate_user'] | session=demo123


Templates found: ['voice_agent_authentication.jinja', 'voice_agent_system.jinja']
Templates found: ['voice_agent_authentication.jinja', 'voice_agent_system.jinja']


In [21]:
agent.tools

[{'type': 'function',
  'function': {'name': 'authenticate_user',
   'description': 'Authenticate a user by verifying first name, last name, and phone number.',
   'parameters': {'type': 'object',
    'properties': {'first_name': {'type': 'string',
      'description': "User's first name."},
     'last_name': {'type': 'string', 'description': "User's last name."},
     'phone_number': {'type': 'string',
      'description': "User's phone number (digits only, no spaces)."}},
    'required': ['first_name', 'last_name', 'phone_number'],
    'additionalProperties': False}}}]